In [32]:
import pandas as pd

from keras.models import Sequential
from keras.layers import Dense

In [33]:
index_clean = []
df_list = []
for i in range(10):
    df = pd.read_csv(f"Datasets/Duplicated{i+1}MeanPastMatches.csv")
    df.drop_duplicates(inplace=True)
    df.dropna(inplace=True)
    index_limpio = set(df["index"])

    df_list.append(df)
    index_clean.append(index_limpio)

In [34]:
init_set = index_clean[0]
for i in index_clean:
    init_set = init_set.intersection(i)
final_set = list(init_set)

In [35]:
len(final_set)

62093

In [44]:
# En este caso X_train, X_val y X_test son listas de dataframes
X_train, X_val, X_test = [], [], []
y_train, y_val, y_test = [], [], []
for i in range(10):
    df = df_list[i][df_list[i]["index"].isin(final_set)]
    df = pd.get_dummies(df, columns=["surface", "player1_hand", "player2_hand", "best_of"], dtype=int)
    X = df.drop(["label"],axis = 1)
    y = df["label"]

    X_train_df = X[X["tourney_date"] < 20150101].drop("tourney_date", axis=1) # 2000 - 2014
    X_val_df = X[(X["tourney_date"]>= 20150101) & (X["tourney_date"] < 20190101)].drop("tourney_date", axis=1) # 2015 - 2018
    X_test_df = X[X["tourney_date"] >= 20190101].drop("tourney_date", axis=1) # 2018 - 2024

    y_train_aux, y_val_aux, y_test_aux = y[X_train_df.index], y[X_val_df.index], y[X_test_df.index]
    
    X_train.append(X_train_df)
    X_val.append(X_val_df)
    X_test.append(X_test_df)

    y_train.append(y_train_aux)
    y_val.append(y_val_aux)
    y_test.append(y_test_aux)



    

In [53]:
total = len(X_train[1]) + len(X_val[1]) + len(X_test[1])
print(f"Tamaño de los sets: \n Set training {len(X_train[1])/total} \n Set val {len(X_val[1])/total} \n Set test {len(X_test[1])/total}")


Tamaño de los sets: 
 Set training 0.6465140998180149 
 Set val 0.17191954004477156 
 Set test 0.18156636013721353


In [68]:
# Veamos el desbalance en cada particion, deberia ser totalmente balanceada

print(sum(y_train[0]==1) / len(y_train[0]))

print(sum(y_val[0]==1) / len(y_val[0]))

print(sum(y_test[0]==1) / len(y_test[0]))

0.5
0.5
0.5


# Modelos

In [76]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

range_T = [5*(i+1) for i in range(8)] # Modelos base

for i in range(len(X_train)):
    print(f"\nSet número {i+1}")
    # Por cada info
    set_X_train = X_train[i]
    set_X_val = X_val[i]

    set_y_train = y_train[i]
    set_y_val = y_val[i]

    best_tuple = ()
    best_acc = 0


    for t_ in range_T:
        clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=2), n_estimators=t_, random_state=0)
        clf.fit(set_X_train, set_y_train)
        y_pred = clf.predict(set_X_val)
        acc = metrics.accuracy_score(set_y_val, y_pred)
        if acc > best_acc:
            best_acc = acc
            best_tuple = (2 , t_)
            print(f"Nuevo mejor modelo {best_tuple}, con {best_acc}")
        
    for t_ in range_T:
        clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=3), n_estimators=t_, random_state=0)
        clf.fit(set_X_train, set_y_train)
        y_pred = clf.predict(set_X_val)
        acc = metrics.accuracy_score(set_y_val, y_pred)
        if acc > best_acc:
            best_acc = acc
            best_tuple = (3 , t_)
            print(f"Nuevo mejor modelo {best_tuple}, con {best_acc}")
    for t_ in range_T:
        clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=4), n_estimators=t_, random_state=0)
        clf.fit(set_X_train, set_y_train)
        y_pred = clf.predict(set_X_val)
        acc = metrics.accuracy_score(set_y_val, y_pred)
        if acc > best_acc:
            best_acc = acc
            best_tuple = (4 , t_)
            print(f"Nuevo mejor modelo {best_tuple}, con {best_acc}")



Set número 1
Nuevo mejor modelo (2, 5), con 0.6463700234192038
Nuevo mejor modelo (2, 10), con 0.6492271662763466
Nuevo mejor modelo (2, 15), con 0.6550351288056206

Set número 2
Nuevo mejor modelo (2, 5), con 0.6451053864168619
Nuevo mejor modelo (2, 10), con 0.651615925058548
Nuevo mejor modelo (2, 15), con 0.6523185011709602
Nuevo mejor modelo (2, 25), con 0.6524590163934426
Nuevo mejor modelo (2, 35), con 0.6532084309133489

Set número 3
Nuevo mejor modelo (2, 5), con 0.6441686182669789
Nuevo mejor modelo (2, 10), con 0.6539110070257611
Nuevo mejor modelo (2, 15), con 0.6543325526932084
Nuevo mejor modelo (2, 20), con 0.6549882903981264

Set número 4
Nuevo mejor modelo (2, 5), con 0.6452459016393443
Nuevo mejor modelo (2, 10), con 0.6538173302107728
Nuevo mejor modelo (2, 15), con 0.6556908665105387
Nuevo mejor modelo (2, 20), con 0.6557377049180327
Nuevo mejor modelo (2, 25), con 0.6560187353629977

Set número 5
Nuevo mejor modelo (2, 5), con 0.6488056206088993
Nuevo mejor modelo

In [88]:
# Ahora que tenemos los mejores hiperparámetros, hacemos un modelo a partir de ese 
hiperparameters = {1:(2, 15), 2:(2, 35), 3:(2, 20),
                   4:(2, 25), 5:(2, 15), 6:(2, 20),
                   7:(2, 20), 8:(2, 25), 9:(2, 35),
                   10:(2, 15)}
models = []
for i in range(len(hiperparameters)):
    depth, t_ = hiperparameters[i+1]
    model = AdaBoostClassifier(DecisionTreeClassifier(max_depth=depth), n_estimators=t_, random_state=0)
    model.fit(X_train[i], y_train[i])
    models.append(model)


In [89]:
# Ahora, hacemos el meta-ensamble:
X = []
for model in models:
    prediction = model.predict(set_X_train)
    X.append(prediction)

